# CINEMA-OT Analysis of GL261 Data

This notebook contains code for generating and plotting a CINEMA-OT synergy analysis. It heavily references the CINEMA-OT tutorial provided by the method authors [here](https://github.com/vandijklab/CINEMA-OT/blob/main/cinemaot_tutorial.ipynb).

Inputs:
- `adata` objects containing the preprocessed single cell expression data for each of the 3 experimental contexts. These were converted from Seurat objects using the `SeuratDisk` package's `SaveH5Seurat` and `Convert` functions.
- Metadata on screening phenotypes for each of the perturbations

Outputs:
- A synergy plot displaying synergy for each perturbation shared across the 3 contexts

## Setup

Running this notebook requires the imported packages and their dependencies to be installed. Just like the others, `cinemaot` can be installed via `pip`. A complete `requirements.txt` for the environment used to run these analyses is provided in the repository as well.

In [ ]:
# Imports
import numpy as np
import scanpy as sc
import cinemaot as co
from cinemaot import utils
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn.metrics

# Plotting configurations
plt.rcParams["font.family"] = "Arial"
plt.rcParams["figure.dpi"] = 250
pd.options.display.max_seq_items = 2000

## Preprocessing

In [ ]:
# Define constants
CONTEXTS = ["invitro", "CED", "preinf"]

# Import the data
raw_adatas = {}
for context in CONTEXTS:
    raw_adatas[context] = sc.read_h5ad(f"input_data/gl261_{context}_preprocessed.h5ad")

Run QC on each `adata` object: remove perturb x radiation combinations with low coverage (<= 5 cells), non-targeting guides we don't wish to use, and return an adata object with the processed object.

In [ ]:
def run_qc(adata):
    adata = adata[~adata.obs["sgRNA"].isin(["NA", "non-targeting_B"])]
    sgRNACond_counts = adata.obs["sgRNACond"].value_counts()
    sgRNAConds_to_keep = sgRNACond_counts[sgRNACond_counts > 5].index
    cell_count_filtered_data = adata[adata.obs["sgRNACond"].isin(sgRNAConds_to_keep), :]
    print("Removing ", sgRNACond_counts[sgRNACond_counts <= 5].index)

    sgRNAs = pd.unique(cell_count_filtered_data.obs.sgRNA)
    sgRNA_conds_in_data = pd.unique(cell_count_filtered_data.obs.sgRNACond)
    sgRNAs_to_keep = []
    for sgRNA in sgRNAs:
        if f"{sgRNA}_RT" in sgRNA_conds_in_data and f"{sgRNA}_noRT" in sgRNA_conds_in_data:
            sgRNAs_to_keep.append(sgRNA)
    RT_condition_filtered_data = adata[adata.obs["sgRNA"].isin(sgRNAs_to_keep), :]
    return RT_condition_filtered_data

In [ ]:
adatas = {}
for context, adata in raw_adatas.items():
    adatas[context] = run_qc(adata)

Separate highly variable genes and run PCA on each `adata` object. Visualize the results in UMAP space

In [ ]:
def run_pca_and_plot_umap(adata, context):
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    adata = adata[:, adata.var.highly_variable]
    sc.tl.pca(adata, svd_solver="arpack")
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
    sc.tl.umap(adata)
    return adata

In [ ]:
for context, adata in adatas.items():
    adatas[context] = run_pca_and_plot_umap(adata, context)

## CINEMA-OT Analysis

For each perturbation in each context, create a synergy score matrix that displays synergy per gene per control cell.

In [10]:
def find_synergy_matrices(adata):
    synergy_matrices = {}
    sgRNAs = pd.unique(adata.obs.sgRNA)
    for sgRNA in sgRNAs:
        if sgRNA == "non-targeting":
            continue
        else:
            synergy_matrix = co.cinemaot.synergy(
                adata,
                "sgRNACond",
                "non-targeting_noRT",
                f"{sgRNA}_noRT",
                "non-targeting_RT",
                f"{sgRNA}_RT",
                dim = 10,
                thres = 0.5,
                smoothness = 1e-3,
                eps = 1e-3,
                mode = "parametric",
            )
            synergy_matrices[sgRNA] = synergy_matrix
    return synergy_matrices

In [ ]:
synergy_matrices_by_context = {}
for context in CONTEXTS:
    adata = adatas[context]
    synergy_matrices_by_context[context] = find_synergy_matrices(adata)

For each matrix, calculate the L2 norm of synergies for genes whose abs(average synergies) across cells is greater than 0.15.

In [ ]:
for context in synergy_matrices_by_context:
    synergy_matrices = synergy_matrices_by_context[context]
    for name, matrix in synergy_matrices.items():
        synergyscore = np.linalg.norm(matrix.X[:,np.abs(np.mean(matrix.X,axis=0))>0.15],axis=1)
        matrix.obs['synergy_l2_norm_squared'] = synergyscore

Plot the perturbations and their synergy scores per cell. Overlay the screening phenotypes on top and compare the three contexts.

In [ ]:
# Identify the perturbations that are shared between all contexts
shared_perturbations = set.intersection(*(set(synergy_matrices.keys()) for _, synergy_matrices in synergy_matrices_by_context.items()))

In [ ]:
# Construct a series of dataframes that include perturbation, synergy, and context information
df_list = []
for context, synergy_matrices in synergy_matrices_by_context.items():
    for perturbation, adata in synergy_matrices.items():
        if perturbation in shared_perturbations:
            temp_df = pd.DataFrame({
                'Perturbation': perturbation,
                'Synergy': adata.obs['synergy_l2_norm'],
                'Context': context
            })
            df_list.append(temp_df)

# Concatenate all the data frames
combined_df = pd.concat(df_list)

In [ ]:
# Create a sorting key so we can sort by the experimental context
order = ["invitro", "preinf", "CED"]

def sort_key(value):
    if value in order:
        return order.index(value)
    else:
        return len(order) + 1

# Apply the custom sorting key function to the column and sort by it
combined_df = combined_df.iloc[combined_df['Context'].map(sort_key).argsort()]

In [ ]:
# Import the screen phenotype metadata we want to overlay on top
screen_phenotypes = pd.read_csv('screen_phenotypes.txt', sep='\t')

In [ ]:
# Plot the synergy values ordered by the median CED L2 norm of synergy
median_synergy = combined_df[combined_df['Context'] == 'CED'].groupby('Perturbation')['Synergy'].median().sort_values()
perturbation_order = median_synergy.index.tolist()

# Create the screening phenotype values
screen_phenotype_values = {}
for screen_phenotype in ['gamma', 'rho', 'tau']:
    values = [screen_phenotypes.at[perturb, screen_phenotype] if perturb in screen_phenotypes.index else 0 for perturb in perturbation_order]
    screen_phenotype_values[screen_phenotype] = values

# Create a figure and a set of subplots
fig, (ax_gamma, ax_rho, ax_tau, ax_violin) = plt.subplots(4, 1, figsize=(15, 8), gridspec_kw={'height_ratios': [1, 1, 1, 4]}, sharex=True)
# phenotype_yticks = [-0.2, 0, 0.2]

# Create the bar plot with gamma values on the top, then rho and tau
bar_width = 1.0
ax_gamma.bar(perturbation_order, screen_phenotype_values['gamma'], color="gray", width=bar_width)
ax_rho.bar(perturbation_order, screen_phenotype_values['rho'], color="gray", width=bar_width)
ax_tau.bar(perturbation_order, screen_phenotype_values['tau'], color="gray", width=bar_width)

ax_gamma.set_ylabel("Gamma")
ax_rho.set_ylabel("Rho")
ax_tau.set_ylabel("Tau")

# Create the violin plot with the synergy scores on the bottom
sns.violinplot(ax=ax_violin, x='Perturbation', y='Synergy', hue='Context', data=combined_df, 
               inner=None, bw=0.2, cut=0, scale='width', order=perturbation_order)
sns.stripplot(ax=ax_violin, x='Perturbation', y='Synergy', hue = 'Context', data=combined_df, dodge = True, color='k', alpha=0.5, size=0.5, jitter=True, order=perturbation_order)
ax_violin.set_ylabel("L2-Norm of Synergy")

# Rotate the X-axis labels for better visibility
plt.xticks(rotation=90)

# Adjust layout for better spacing
plt.tight_layout()

# Save the figure
plt.savefig(f'output_plots/combined_aligned_phenotypes_and_synergy_plot_ordered_by_CED_median_l2Norm.pdf', format='pdf', bbox_inches='tight')

# Show the plot
plt.show()